In [1]:
import os
from PIL import Image
import torch
import numpy as np
import cv2
from DPR.utils.utils_SH import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = "cuda"

# Extract Lighting
using https://github.com/zhhoper/DPR

In [2]:
from DPR.DPR import DPR
dpr = DPR(device)

In [3]:
modelFolder = 'DPR/trained_model/'

# load model
from DPR.model.defineHourglass_1024_gray_skip_matchFeature import *
my_network_512 = HourglassNet(16)
my_network = HourglassNet_1024(my_network_512, 16)
my_network.load_state_dict(torch.load(os.path.join(modelFolder, 'trained_model_1024_03.t7')))
my_network.cuda()
my_network.train(False)

HourglassNet_1024(
  (model_512): HourglassNet(
    (pre_conv): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (pre_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (light): lightingNet(
      (predict_FC1): Conv2d(27, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (predict_relu1): PReLU(num_parameters=1)
      (predict_FC2): Conv2d(128, 9, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (post_FC1): Conv2d(9, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (post_relu1): PReLU(num_parameters=1)
      (post_FC2): Conv2d(128, 27, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (post_relu2): ReLU()
    )
    (HG0): HourglassBlock(
      (upper): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0

In [5]:
i = 0
image_path = os.path.join("./example_images", 'obama_relighted_{:02d}.jpg'.format(i))
image = Image.open(image_path).convert("RGB")
img = (torch.from_numpy(np.array(image)).float().permute(2, 0, 1)[None, ...]/127.5-1).to(device)
sh = dpr.extract_lighting(img).squeeze()[None, ...]

In [6]:
sh

tensor([[ 0.7496, -0.1781,  0.0714,  0.2698, -0.1625,  0.0797, -0.2537,  0.0032,
          0.0416]], device='cuda:0')

# Decompose
using https://github.com/andrewhou1/GeomConsistentFR

In [8]:
from GeomConsistentFR.RelightNet import RelightNet
import imageio

model = RelightNet()
model.load_state_dict(torch.load('GeomConsistentFR/model/model_epoch99.pth'))
model = model.float()
model = model.cuda()
model.eval()
training_images = np.zeros((1, 256, 256, 3))

In [10]:
training_images[0, :, :, :] = cv2.resize(imageio.imread(image_path)/255.0, (256, 256))
training_lightings = np.zeros((1, 4))
training_lightings[0, 0] = 0.5
training_lightings[0, 1] = 0
training_lightings[0, 2] = 0.7071
training_lightings[0, 3] = 0.7071

training_masks = np.zeros((1, 256, 256, 1))
training_masks.fill(255)
training_masks_fill_nose = np.zeros((1, 256, 256, 1))
training_masks_fill_nose.fill(255)
epoch = 200
intrinsic_matrix = np.zeros((1, 3, 3))
intrinsic_matrix[:, 0, 0] = 1570.0
intrinsic_matrix[:, 1, 1] = 1570.0
intrinsic_matrix[:, 2, 2] = 1.0
intrinsic_matrix[:, 0, 2] = model.img_width/2.0
intrinsic_matrix[:, 1, 2] = model.img_height/2.0
intrinsic_matrix = torch.from_numpy(intrinsic_matrix)
batch_list = np.arange(1)
num_batches = 1

L1_loss = nn.L1Loss()
L1_loss_sum = nn.L1Loss(reduction='sum')

with torch.no_grad():
    for j in range(num_batches):
        curr_input_images = torch.from_numpy(training_images[(batch_list[j]*model.batch_size):((batch_list[j]+1)*model.batch_size)]) 
        curr_training_lightings = torch.from_numpy(training_lightings[(batch_list[j]*model.batch_size):((batch_list[j]+1)*model.batch_size)])
        curr_mask = torch.from_numpy(training_masks[batch_list[j]])/255.0
        batch_mask = curr_mask.repeat(model.batch_size, 1, 1, 1)
        albedo, depth, shadow_mask_weights, ambient_light, full_shading, rendered_images, unit_light_direction, ambient_values, final_shading, surface_normals = model(curr_input_images.float().cuda(), epoch, intrinsic_matrix.cuda(), curr_mask.cuda(), torch.reshape(curr_training_lightings[:, 1:4].float().cuda(), (model.batch_size, 3, 1, 1)), torch.reshape(curr_training_lightings[:, 0].float().cuda(), (model.batch_size, 1, 1)), batch_mask.cuda())
        
        rendered_images = rendered_images.permute(0, 2, 3, 1)
        rendered_images = rendered_images.cpu().numpy()
        albedo = albedo.permute(0, 2, 3, 1)
        albedo = albedo.cpu().numpy()
        depth = depth.permute(0, 2, 3, 1)
        depth = depth.cpu().numpy()
        depth = -depth
        depth = (depth-np.amin(depth))/(np.amax(depth)-np.amin(depth))
        final_shading = final_shading.cpu().numpy()
        surface_normals = surface_normals.permute(0, 2, 3, 1)
        surface_normals = surface_normals.cpu().numpy()
        surface_normals = 255.0*(surface_normals+1.0)/2.0

        cv2.imwrite(image_path.split('.jpg')[0] + '_albedo.png', albedo[0].clip(0, 1)[:, :, ::-1]*255.0)
        cv2.imwrite(image_path.split('.jpg')[0] + '_depth.png', depth[0].clip(0, 1)[:, :, ::-1]*255.0)
        cv2.imwrite(image_path.split('.jpg')[0] + '_shading.png', final_shading[0].clip(0, 1)*255.0)
        cv2.imwrite(image_path.split('.jpg')[0] + '_normal.png', surface_normals[0].clip(0, 255)[:, :, ::-1])

<ipython-input-10-4ce4ad642e70>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  training_images[0, :, :, :] = cv2.resize(imageio.imread(image_path)/255.0, (256, 256))
